In [ ]:
from indexify import IndexifyClient, ExtractionGraph

In [ ]:
client = IndexifyClient()

In [ ]:
extraction_graph_spec = """
name: 'pdflearner'
extraction_policies:
   - extractor: 'tensorlake/paddleocr_extractor'
     name: 'pdf_to_text'
"""

In [ ]:
extraction_graph = ExtractionGraph.from_yaml(extraction_graph_spec)
# client.create_extraction_graph(extraction_graph)

In [ ]:
content_id = client.upload_file("pdflearner", "/home/deepkapha/project/quamer/faster-patient-onboarding-from-medical-history-using-indexify/data/BalanceSheet.pdf")

In [ ]:
client.wait_for_extraction(content_id)

In [ ]:
extracted_content = client.get_extracted_content(ingested_content_id=content_id, graph_name="pdflearner", policy_name="pdf_to_text")

In [ ]:
extracted_content

In [ ]:
schema = {
'properties': {
    'name': {'title': 'Name', 'type': 'string'},
    'dob': {'title': 'Date of Birth', 'type': 'string'},
    'problem_description': {'title': 'Description of Problem or Complaints', 'type': 'string'},
    'accident_type': {'title': 'Is the present problem due to an injury?', 'type': 'string'},
    'accident_reported': {'title': 'Has the accident been reported?', 'type': 'string'},
    'family_history': {'title': 'Family History', 'type': 'string'},
    'job_description': {'title': 'Job Description', 'type': 'string'},
    'genral_symptoms': {'title': 'General Symptoms', 'type': 'string'},
    'muscles_and_joints_symptoms': {'title': 'Muscles and Joints Symptoms', 'type': 'string'},
    'cardiovascular_symptoms': {'title': 'Cardiovascular Symptoms', 'type': 'string'},
},
'required': ['name', 'dob', 'problem_description', 'accident_type', 'accident_reported', 'family_history', 'job_description', 'genral_symptoms', 'muscles_and_joints_symptoms', 'cardiovascular_symptoms'],
'title': 'User',
'type': 'object',
}



extraction_graph_spec = f"""
name: 'invoice-learner2'
extraction_policies:
  - extractor: 'tensorlake/paddleocr_extractor'
    name: 'pdf-extraction'
  - extractor: 'tensorlake/schema'
    name: 'text_to_json'
    input_params:
      service: 'openai'
      key: ''
      schema_config : {schema}
      additional_messages: 'Extract information in JSON according to this schema and return only the output. You need to provide the schema as given in the context.'
    content_source: 'pdf-extraction'
"""

In [ ]:
# from pydantic import BaseModel
# class Invoice(BaseModel):
#     year: int
#     captial_deposit: int
#     total_liability: int
#     total_assests: int
#     ratio_of_assests_by_liabilities: float
#     bills_collection: int
#     increase_in_deposites_from_2021_to_2022: float

# schema = Invoice.model_json_schema()
# schema

In [ ]:
from indexify import IndexifyClient, ExtractionGraph
client = IndexifyClient()

extraction_graph = ExtractionGraph.from_yaml(extraction_graph_spec)
# client.create_extraction_graph(extraction_graph)
content_id = client.upload_file("invoice-learner2", "/home/deepkapha/project/quamer/faster-patient-onboarding-from-medical-history-using-indexify/data/6000267600123615133.pdf")
client.wait_for_extraction(content_id)

In [ ]:
extracted_content = client.get_extracted_content(ingested_content_id=content_id, graph_name="invoice-learner2", policy_name="text_to_json")

In [ ]:
content_id

In [ ]:
extracted_content

In [ ]:
llm_content = next(content.data.decode("utf-8") for content in extracted_content if content.mime_type == "text/plain")
llm_content

In [ ]:
print(extracted_content)

In [ ]:
from indexify_extractor_sdk import Content

In [ ]:
schema = {'properties': {'invoice_number': {'title': 'Invoice Number', 'type': 'string'}, 'date': {'title': 'Date', 'type': 'string'}, 'account_number': {'title': 'Account Number', 'type': 'string'}, 'owner': {'title': 'Owner', 'type': 'string'}, 'address': {'title': 'Address', 'type': 'string'}, 'last_month_balance': {'title': 'Last Month Balance', 'type': 'string'}, 'current_amount_due': {'title': 'Current Amount Due', 'type': 'string'}, 'registration_key': {'title': 'Registration Key', 'type': 'string'}, 'due_date': {'title': 'Due Date', 'type': 'string'}}, 'required': ['invoice_number', 'date', 'account_number', 'owner', 'address', 'last_month_balance', 'current_amount_due', 'registration_key', 'due_date'], 'title': 'User', 'type': 'object'}

data = Content.from_text('Axis\nSTATEMENTInvoice No. 20240501-336593\nDate: 4/19/2024\nAccount Number:\nOwner:\nProperty:922000203826\nJohn Doe\n200 Park Avenue, Manhattan\nJohn Doe\n200 Park Avenue Manhattan\nNew York 10166SUMMARY OF ACCOUNT\nLast Month Balance:\nCurrent Amount Due:$653.03\n$653.03\nAccount details on back.\nProfessionally\nprepared by:\nSTATEMENT MESSAGE\nWelcome to Action Property Management! We are excited to be\nserving your community. Our Community Care team is more than\nhappy to assist you with any billing questions you may have. For\ncontact options, please visit www.actionlife.com/contact. Visit the\nAction Property Management web page at: www.actionlife.com.BILLING QUESTIONS\nScan the QR code to\ncontact our\nCommunity Care\nteam.\nactionlife.com/contact\nCommunityCare@actionlife.com\nRegister your Resident\nPortal account now!\nRegistration Key/ID:\nFLOWR2U\nresident.actionlife.com\nTo learn more about issues facing HOAs, say "Hey Siri, search the web for The Uncommon Area by Action Property Management."\nMake checks payable to:\nAxisAccount Number: 922000203826\nOwner: John Doe\nPLEASE REMIT PAYMENT TO:\n** AUTOPAY SCHEDULED **\n** NO REMITTANCE NECESSARY **CURRENT AMOUNT DUE\n$653.03\nDUE DATE\n5/1/2024\n0049 00008330 0000922000203826 7 00065303 00000000 9')


In [ ]:
import base64
import io
import mimetypes
import os
from typing import Any, Dict, List, Optional, Union

from indexify_extractor_sdk import Content, Extractor, Feature
from openai import OpenAI
from pdf2image import convert_from_bytes
from pydantic import BaseModel, Field


class SchemaExtractorConfig(BaseModel):
    model: Optional[str] = Field(default="gpt-4o-2024-08-06")
    api_key: Optional[str] = Field(default=None)
    system_prompt: str = Field(default="Extract the information.")
    user_prompt: Optional[str] = Field(default=None)
    response_format: Optional[Dict[str, Any]] = Field(
        default={
            "properties": {"name": {"title": "Name", "type": "string"}},
            "required": ["name"],
            "title": "Event",
            "type": "object",
            "additionalProperties": False,
        }
    )


class SchemaExtractor(Extractor):
    name = "tensorlake/schema"
    description = "An extractor that let's you extract JSON from schemas."
    system_dependencies = []
    input_mime_types = [
        "text/plain",
        "application/json",
        "application/pdf",
        "image/jpeg",
        "image/png",
    ]

    def __init__(self):
        super(SchemaExtractor, self).__init__()

    def extract(
        self, content: Content, params: SchemaExtractorConfig
    ) -> List[Union[Feature, Content]]:
        contents = []
        model_name = params.model
        key = params.api_key
        prompt = params.system_prompt
        query = params.user_prompt
        response_format = params.response_format

        if content.content_type == "application/pdf":
            print("1")
            images = convert_from_bytes(content.data)

            all_responses = []
            for image in images:
                img_byte_arr = io.BytesIO()
                image.save(img_byte_arr, format="JPEG")
                img_byte_arr = img_byte_arr.getvalue()
                response = self._process_image(
                    img_byte_arr, model_name, key, prompt, query, response_format
                )
                all_responses.append(response)

            response_content = "\n\n".join(all_responses)

        elif content.content_type in ["image/jpeg", "image/png"]:
            print("2")
            response_content = self._process_image(
                content.data, model_name, key, prompt, query, response_format
            )

        else:
            print("3")
            text = content.data.decode("utf-8")
            if query is None:
                query = text
            response_content = self._process_text(
                model_name, key, prompt, query, response_format
            )

        contents.append(Content.from_text(response_content))
        return contents

    def _process_text(self, model_name, key, prompt, query, response_format):
        client = self._get_client(key)

        messages_content = [
            {"role": "system", "content": prompt},
            {"role": "user", "content": query},
        ]

        try:
            response = client.beta.chat.completions.parse(
                model=model_name,
                messages=messages_content,
                response_format={
                    "type": "json_schema",
                    "json_schema": {
                        "name": "schema_response",  # Add a required name
                        "strict": True,
                        "schema": response_format,
                    },
                },
            )
            return response.choices[0].message.content
        except Exception as e:
            print(f"Unable to process text: {str(e)}")
            raise e

    def _get_client(self, key):
        if ("OPENAI_API_KEY" not in os.environ) and (key is None):
            raise ValueError(
                "The OPENAI_API_KEY environment variable is not present and no API key was provided."
            )
        if ("OPENAI_API_KEY" in os.environ) and (key is None):
            return OpenAI(api_key=os.environ["OPENAI_API_KEY"])
        return OpenAI(api_key=key)

    def sample_input(self) -> Content:
        return Content.from_text("Alice and Bob are going to a science fair on Friday.")

In [ ]:
import os

os.environ["OPENAI_API_KEY"] = " "

In [ ]:

input_params = SchemaExtractorConfig(service="openai", schema=schema)

extractor = SchemaExtractor()
results = extractor.extract(data, params=input_params)

In [ ]:
results[0]

In [ ]:
from indexify import IndexifyClient, ExtractionGraph

In [ ]:
from indexify_extractor_sdk import Content
from pydantic import BaseModel

# PDF extractor initialization
# pdfextractor, pdfconfig_cls = load_extractor("indexify_extractors.pdfextractor.pdf_extractor:PDFExtractor")

# extract content from PDF file

# config = pdfconfig_cls()

# pdf_result = pdfextractor.extract(content, config)

# # text content present in PDF
# text_content = next(content.data.decode('utf-8') for content in pdf_result if content.content_type == 'text/plain')

In [ ]:
schema = {
'properties': {
    'money': {'title': 'Money', 'type': 'string'},
},
'required': ['year',],
'title': 'User',
'type': 'object',
"additionalProperties": False,
}

extraction_graph_spec = f"""
name: 'pdf_schema_extractor'
extraction_policies:
  - extractor: 'tensorlake/paddleocr_extractor'
    name: 'pdf_to_text'
  - extractor: 'tensorlake/schema'
    name: 'text_to_schema'
    input_params:
      service: 'openai'
      model_name: 'gpt-3.5-turbo'
      key: ''
      schema_config: {schema}
      example: {schema}
      data: "Balance SheetAs at March 31, 2022 ` in '000 Schedule As at March 31, 2022 As at March 31, 2021 CAPITAL AND LIABILITIES Capital 1 5,545,541 5,512,776 Reserves and surplus 2 2,395,383,846 2,031,695,513 Deposits 3 15,592,174,400 13,350,602,208 Borrowings 4 1,848,172,073 1,354,873,236 Other liabiliti"
      additional_messages: 'Extract information in JSON according to this schema and return only the output.'
    content_source: 'pdf_to_text'
"""
client = IndexifyClient()
extraction_graph = ExtractionGraph.from_yaml(extraction_graph_spec)
# result = client.create_extraction_graph(extraction_graph)
content_id = client.upload_file("pdf_schema_extractor", "/home/deepkapha/project/quamer/faster-patient-onboarding-from-medical-history-using-indexify/data/BalanceSheet.pdf")


In [ ]:
extraction_graph.to_dict()

In [ ]:
client.wait_for_extraction(content_id)

In [ ]:
extracted_content = client.get_extracted_content(ingested_content_id=content_id, graph_name="pdf_schema_extractor", policy_name="text_to_schema")
extracted_content